## Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import re
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima

## Importanto e tratando os dados

In [ ]:
url = './covid_19_data.csv'

In [ ]:
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])
df

In [ ]:
df.dtypes

In [ ]:
def corrige_colunas(col_name):
  return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
df.columns = [corrige_colunas(col) for col in df.columns]

## Selecionando dados do Brasil

In [ ]:
df.loc[df.countryregion == 'Brazil']

In [ ]:
brasil = df.loc[
    (df.countryregion == 'Brazil') &
    (df.confirmed > 0)
]

brasil.head()

## Casos confirmados

In [ ]:
# Gráfico da evolução dos casos confirmados
px.line(brasil, 'observationdate', 'confirmed', title='casos confirmados')

## Casos por dia

In [ ]:
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))

In [ ]:
px.line(brasil, 'observationdate', 'novoscasos', title='novos casos')

## Casos de mortes

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=brasil.observationdate, y=brasil.deaths, name='Mortes',
               mode='lines+markers', line={'color':'red'})
)
fig.update_layout(title='Mortes por covid no brasil')


## Taxa de crescimento

In [ ]:
def taxa_crescimento(data, var, data_ini=None, data_fim=None):
  if data_ini == None:
    data_ini = data.observationdate.loc[data[var] > 0].min()
  else:
    data_ini = pd.to_datetime(data_ini)

  if data_fim == None:
    data_fim = data.observationdate.iloc[-1]
  else:
    data_fim = pd.to_datetime(data_fim)

  passado = data.loc[data.observationdate == data_ini, var].values[0]
  presente = data.loc[data.observationdate == data_fim, var].values[0]

  n = (data_fim - data_ini).days

  taxa = (presente/passado)**(1/n) - 1

  return taxa*100

In [ ]:
taxa_crescimento(brasil, 'confirmed')

In [ ]:
def taxa_crescimento_diaria(data, var, data_ini=None):
  if data_ini == None:
    data_ini = data.observationdate.loc[data[var] > 0].min()
  else:
    data_ini = pd.to_datetime(data_ini)

  data_fim = data.observationdate.max()
  n = (data_fim - data_ini).days

  taxas = list(map(
      lambda x: (data[var].iloc[x] - data[var].iloc[x-1]) / data[var].iloc[x-1],
      range(1, n+1)
  ))

  return np.array(taxas) * 100

In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')
tx_dia

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:], y=tx_dia, title='Taxa de crescimento por dia')

## Predições

In [ ]:
confirmados = brasil.confirmed
confirmados.index = brasil.observationdate
confirmados

In [ ]:
res = seasonal_decompose(confirmados)

In [ ]:
# fig, axes = plt.subplots(4, 1, figsize=(10,8))
fig, (ax1,ax2,ax3,ax4) = plt.subplots(4, 1, figsize=(10,8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, c='k', linestyle='--')
plt.show()

## ARIMA

In [ ]:
modelo = auto_arima(confirmados)

In [ ]:
fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados, name="Observados"
))

fig.add_trace(go.Scatter(
    x=confirmados.index, y=modelo.predict_in_sample(), sample='Preditos'
))

fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20', '2020-06-20'), y=modelo.predict(31), sample='Forecast'
))

fig.update_layout(title='Previsão de casos confirmados no Brasil')

## Modelo de crescimento

In [ ]:
from fbprophet import Prophet

In [ ]:
train = confirmados.reset_index()[: -5]
test = confirmados.reset_index()[-5:]

train.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)
test.rename(columns={'observationdate':'ds', 'confirmed': 'y'}, inplace=True)

profeta = Prophet(growth='logistic', changepoints=['20-03-21', '2020-03-30', '2020-04-25',
                                                  '2020-05-03', '2020-05-10'])


pop = 211463256
train['cap'] = pop

profeta.fit(train)

future_dates = profeta.make_future_dataframe(periods=200)
future_dates['cap'] = pop
forecast = profeta.predict(future_dates)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=test.index, y=test, name='Observados - Teste'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')